In [1]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import json

import time # importing time module

In [2]:
with open('APIkeys.json') as f:
    keys = json.load(f)
    omdbapi = keys['OMDBapi']

In [3]:
serviceurl = 'http://www.omdbapi.com/?'
apikey = '&apikey='+omdbapi

## Function for printing JSON dataset

In [4]:
def print_json(json_data):
    list_keys=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Ratings', 
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID']
    print("-"*50)
    for k in list_keys:
        if k in list(json_data.keys()):
            print(f"{k}: {json_data[k]}")
    print("-"*50)

## Function to download poster

In [5]:
def save_poster(json_data):
    import os
    title = json_data['Title']
    poster_url = json_data['Poster']
    # Splits the poster url by '.' and picks up the last string as file extension
    poster_file_extension=poster_url.split('.')[-1]
    # Reads the image file from web
    poster_data = urllib.request.urlopen(poster_url).read()
        
    savelocation=os.getcwd()+'\\'+'Posters'+'\\'
    # Creates new directory if the directory does not exist. Otherwise, just use the existing path.
    if not os.path.isdir(savelocation):
        os.mkdir(savelocation)
    
    filename=savelocation+str(title)+'.'+poster_file_extension
    f=open(filename,'wb')
    f.write(poster_data)
    f.close()

## Function to create/update the local movie database with the data retreived from the web


Saves the movie data (Title, Year, Runtime, Country, Metascore, and IMDB rating) into a local SQLite database called 'movies.sqlite'

In [6]:
#this is the query string
#http://www.omdbapi.com/?t=Toy+Story&y=1995

In [7]:
def save_in_database(json_data):
    
    filename = 'output/IMDB_tv.sqlite'
    
    import sqlite3
    conn = sqlite3.connect(str(filename))
    cur=conn.cursor()
    
    title = json_data['Title']
    # Goes through the json dataset and extracts information if it is available
    if json_data['Year']!='N/A':
        year = json_data['Year']
    else:
        year='NA'
    if json_data['Rated']!='N/A':
        rated = json_data['Rated']
    else:
        rated='NA'
    if json_data['Released']!='N/A':
        released = json_data['Released']
    else:
        released='NA'
    if json_data['Runtime']!='N/A':
        runtime = int(json_data['Runtime'].split()[0])
    else:
        runtime=-1
        
    if json_data['Genre']!='N/A':
        genre = json_data['Genre']
    else:
        genre='NA'   
        
    if json_data['Director']!='N/A':
        director = json_data['Director']
    else:
        director='NA'   
        
    if json_data['Writer']!='N/A':
        writer = json_data['Writer']
    else:
        writer='NA'   
                
    if json_data['Actors']!='N/A':
        actors = json_data['Actors']
    else:
        actors='NA' 
        
    if json_data['Language']!='N/A':
        language = json_data['Language']
    else:
        language='NA'       

    if json_data['Country']!='N/A':
        country = json_data['Country']
    else:
        country='NA'       
        
    if json_data['Awards']!='N/A':
        awards = json_data['Awards']
    else:
        awards='NA'            
    if json_data['Metascore']!='N/A':
        metascore = float(json_data['Metascore'])
    else:
        metascore=-1
    if json_data['imdbRating']!='N/A':
        imdb_rating = float(json_data['imdbRating'])
    else:
        imdb_rating=-1
    if json_data['imdbVotes']!='N/A':
        imdb_votes = json_data['imdbVotes']
    else:
        imdb_votes=-1
    if json_data['imdbID']!='N/A':
        imdbId = json_data['imdbID']
        
    
    # SQL commands
    cur.execute('''CREATE TABLE IF NOT EXISTS MovieInfo 
    (Title TEXT, Year TEXT, Rated TEXT, Released TEXT, Runtime INTEGER, Genre TEXT, Director TEXT, Writer TEXT, Actors TEXT, Language TEXT, Country TEXT, Awards TEXT, Metascore REAL, IMDBRating REAL, imdbVotes TEXT, imdbID TEXT)''')
    
    cur.execute('SELECT Title FROM MovieInfo WHERE Title = ? ', (title,))
    row = cur.fetchone()
    
    if row is None:
        cur.execute('''INSERT INTO MovieInfo (Title, Year, Rated, Released, Runtime, Genre, Director, Writer, Actors, Language, Country, Awards, Metascore, IMDBRating, imdbVotes, imdbID )
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', (title,year,rated,released,runtime, genre, director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes,imdbId))
    else:
        print("Record already found. No update made.")
    
    # Commits the change and close the connection to the database
    conn.commit()
    conn.close()

## Function to print contents of the local database

In [8]:
def print_database(database):
    
    import sqlite3
    conn = sqlite3.connect(str(database))
    cur=conn.cursor()
    
    for row in cur.execute('SELECT * FROM MovieInfo'):
        print(row)
    conn.close()

## Function to save the database content in an CSV file

In [9]:
def save_in_csv(filename, database):
    
    if filename.split('.')[-1]!='csv':
        print ("Filename does not have correct extension. Please try again")
        return None
    
    import pandas as pd
    import sqlite3
    
    conn = sqlite3.connect(str(database))
    #cur=conn.cursor()
    
    df=pd.read_sql_query("SELECT * FROM MovieInfo", conn)
    conn.close()
    
    df.to_csv(filename, index = None, header=True)

## Function to search for information about a movie

In [10]:
def search_movie(m_title,year):
    if len(title) < 1 or title=='quit': 
        print("Goodbye now...")
        return None

    try:
        url = serviceurl + urllib.parse.urlencode({'t': title})+ '&' + urllib.parse.urlencode({'y': year}) + apikey
        
        print(f'Retrieving the data of "{title}" now... ')
        uh = urllib.request.urlopen(url)
        data = uh.read()
        json_data=json.loads(data)
        
        if json_data['Response']=='True':
            print_json(json_data)
            save_in_database(json_data)
            
        else:
            try:
                
                #try again this time with only 2 first characters from the movie title
            
                first = title.split(' ')[0].lower()
                first = first.ljust(len(first)+1)
                second = title.split(' ')[1].lower()
                m_title = first+second
                print(m_title)
                print(year)
                #title = first + ' '+ second
                print('--------2nd try: 2 words title --------')

                url = serviceurl + urllib.parse.urlencode({'t': m_title})+ '&' + urllib.parse.urlencode({'y': year}) + apikey
                time.sleep(5)
                save_in_database(json_data)
            except:
                print("Error: ",json_data['Error'])
            
    
    except urllib.error.URLError as e:
        print(f"ERROR: URL Error {e.reason}")

## Load Disney titles to get all the information through function search_movie

In [11]:
#Load Disney titles

disney_df = pd.read_csv('output/2021_disney_plus_shows.csv')
# disney_df.drop(columns=['release_date','total_gross','inflation_adjusted_gross','_merge',\
#                         'director','cast','country','description','date_added'], inplace=True)
disney_df.head()

,show_id,type,movie_title,director,cast,country,date_added,year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
2,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
3,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...
4,s6,Movie,Becoming Cousteau,Liz Garbus,"Jacques Yves Cousteau, Vincent Cassel",United States,"November 24, 2021",2021,PG-13,94 min,"Biographical, Documentary",An inside look at the legendary life of advent...


In [12]:
disney_df.count()

show_id        1226
type           1226
movie_title    1226
director        753
cast           1037
country        1008
date_added     1223
year           1226
rating         1223
duration       1226
listed_in      1226
description    1226
dtype: int64

## === Test only STARTS here ===

In [13]:
# #test with this df to see if it works as free API call is only 1000 calls per day
# test_df = disney_df.loc[disney_df['movie_title'].isin(['101 Dalmatians','1492: Conquest of Paradise'])]
# test_df

In [14]:
# #TEST BLOCK: For smaller DF due to 1000 triggers a day only
# #query for all the movies in our disney list and get the latest information from IMDB

# for i in test_df.index: 
#     title = test_df['movie_title'][i].lower()
#     year = test_df['year'][i]
#     print (test_df['movie_title'][i])
#     print (test_df['year'][i])
#     print ('=====================================')
#     search_movie(title,year)
#     time.sleep(10)

## === Test only ENDS here ===

In [18]:
#FIRST HALF QUERY
#query for all the tv shows in our list and get the latest information from IMDB
#half the list due to 1000 calls / day limit

for i in range(0,35): 
    title = disney_df['movie_title'][i].lower()
    year = disney_df['year'][i]
    print (disney_df['movie_title'][i])
    print (disney_df['year'][i])
    print ('=====================================')
    search_movie(title,year)
    time.sleep(10)

Duck the Halls: A Mickey Mouse Christmas Special
2016
Retrieving the data of "duck the halls: a mickey mouse christmas special" now... 
duck the
2016
--------2nd try: 2 words title --------
Error:  Movie not found!
Ice Age: A Mammoth Christmas
2011
Retrieving the data of "ice age: a mammoth christmas" now... 
--------------------------------------------------
Title: Ice Age: A Mammoth Christmas
Year: 2011
Rated: TV-PG
Released: 24 Nov 2011
Runtime: 26 min
Genre: Animation, Short, Adventure, Comedy, Family, Fantasy
Director: Karen Disher
Writer: Sam Harper, Mike Reiss
Actors: Ray Romano, John Leguizamo, Denis Leary, Queen Latifah
Plot: Sid, Eddie and Crash get in trouble with Santa and have to get their names off of the naughty list.
Language: English
Country: USA
Awards: 1 nomination.
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.5/10'}]
Metascore: N/A
imdbRating: 6.5
imdbVotes: 9,671
imdbID: tt2100546
--------------------------------------------------
The Queen Family Si

Dr. Oakley, Yukon Vet
2013
Retrieving the data of "dr. oakley, yukon vet" now... 
dr. oakley,
2013
--------2nd try: 2 words title --------
Error:  Movie not found!
America the Beautiful
2021
Retrieving the data of "america the beautiful" now... 
america the
2021
--------2nd try: 2 words title --------
Error:  Movie not found!
Baymax!
2021
Retrieving the data of "baymax!" now... 
Error:  Movie not found!
Ciao Alberto
2021
Retrieving the data of "ciao alberto" now... 
--------------------------------------------------
Title: Ciao Alberto
Year: 2021
Rated: G
Released: 12 Nov 2021
Runtime: 8 min
Genre: Animation, Short, Comedy
Director: McKenna Harris
Writer: Matthias De Clercq, Brooklyn El-Omar, McKenna Harris
Actors: Jack Dylan Grazer, Marco Barricelli, Jacob Tremblay
Plot: An animated short film that features characters from "Luca".
Language: English, Italian
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.9/10'}]
Metascore: N/A
imdbRating:

Spin
2021
Retrieving the data of "spin" now... 
--------------------------------------------------
Title: Spin
Year: 2021
Rated: N/A
Released: 24 Oct 2021
Runtime: 16 min
Genre: Short, Drama, Romance
Director: Lisa Tedesco
Writer: Lisa Tedesco
Actors: Jahdey Wright, Glenn Ghirardi, Sierra Berkeley Fisher
Plot: When the curtain descends on their final performance of Romeo and Juliet in drama club, high school seniors Abigail and Sky bid a fair adieu to the stage they loved while letting their most deepest desires for one another surface at 
Language: English
Country: United States
Awards: 21 wins & 5 nominations
Ratings: []
Metascore: N/A
imdbRating: N/A
imdbVotes: 12
imdbID: tt13681768
--------------------------------------------------
Tangled Ever After
2012
Retrieving the data of "tangled ever after" now... 
--------------------------------------------------
Title: Tangled Ever After
Year: 2012
Rated: G
Released: 13 Jan 2012
Runtime: 6 min
Genre: Animation, Short, Adventure
Director:

In [43]:
# #FIRST HALF QUERY
# #query for all the tv shows in our list and get the latest information from IMDB
# #half the list due to 1000 calls / day limit

# for i in range(500,700): 
#     title = disney_df['movie_title'][i].lower()
#     year = disney_df['year'][i]
#     print (disney_df['movie_title'][i])
#     print (disney_df['year'][i])
#     print ('=====================================')
#     search_movie(title,year)
#     time.sleep(10)

Born Wild: The Next Generation
2020
Retrieving the data of "born wild: the next generation" now... 
--------------------------------------------------
Title: Born Wild: The Next Generation
Year: 2020
Rated: N/A
Released: 23 Apr 2020
Runtime: N/A
Genre: Documentary
Director: N/A
Writer: N/A
Actors: Robin Roberts
Plot: A celebration of the 50th anniversary of Earth Day, this documentary is a feel good, heartwarming tale of big and small creatures. Chris Hemsworth makes an appearance with a baby koala.
Language: English
Country: USA
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.8/10'}]
Metascore: N/A
imdbRating: 7.8
imdbVotes: 17
imdbID: tt12194646
--------------------------------------------------
America's Funniest Home Videos
1989
Retrieving the data of "america's funniest home videos" now... 
--------------------------------------------------
Title: America's Funniest Home Videos
Year: 1989–
Rated: TV-PG
Released: 26 Nov 1989
Runtime: 30 min
Genre: Comedy, Fa

Tut's Treasures: Hidden Secrets
2018
Retrieving the data of "tut's treasures: hidden secrets" now... 
--------------------------------------------------
Title: Tut's Treasures: Hidden Secrets
Year: 2018
Rated: N/A
Released: 24 Mar 2018
Runtime: N/A
Genre: Documentary, History
Director: N/A
Writer: N/A
Actors: Salima Ikram, Chris Naunton, Aidan Dodson
Plot: Howard Carter's discovery of Tutankhamun's tomb in 1922 made headlines across the world sparking a global frenzy for Ancient Egypt. But over the decades since the find, many of the pharaoh's priceless grave goods have disappeared int
Language: English
Country: United Kingdom
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.4/10'}]
Metascore: N/A
imdbRating: 7.4
imdbVotes: 181
imdbID: tt8174114
--------------------------------------------------
A Tale of Two Critters
1977
Retrieving the data of "a tale of two critters" now... 
--------------------------------------------------
Title: A Tale of Two Critters
Year:

Elmer Elephant
1936
Retrieving the data of "elmer elephant" now... 
--------------------------------------------------
Title: Elmer Elephant
Year: 1936
Rated: Approved
Released: 28 Mar 1936
Runtime: 9 min
Genre: Animation, Short, Comedy
Director: Wilfred Jackson
Writer: Earl Hurd, Bianca Majolie, Ted Sears
Actors: Pinto Colvig, Bernice Hansen, Clarence Nash
Plot: A little elephant is ashamed of his trunk until he saves his tiger girlfriend with it by using it as a fire hose when her house burns.
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.9/10'}]
Metascore: N/A
imdbRating: 6.9
imdbVotes: 1,288
imdbID: tt0027569
--------------------------------------------------
Fish Hooks
2010
Retrieving the data of "fish hooks" now... 
--------------------------------------------------
Title: Fish Hooks
Year: 2010–2014
Rated: TV-G
Released: 28 Aug 2010
Runtime: 22 min
Genre: Animation, Adventure, Comedy
Director: N/A
Writer: Noah Jon

Error:  Movie not found!
The Boy Who Talked to Badgers
1975
Retrieving the data of "the boy who talked to badgers" now... 
--------------------------------------------------
Title: The Boy Who Talked to Badgers
Year: 1975
Rated: N/A
Released: 14 Sep 1975
Runtime: 89 min
Genre: Adventure, Family
Director: Gary Nelson
Writer: Sheldon Stark, Allan Eckert
Actors: Christian Juttner, Carl Betz, Salome Jens
Plot: A young boy gets along better with the animals he befriends around his family's Canadian farm than with the people he lives with.
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.5/10'}]
Metascore: N/A
imdbRating: 5.5
imdbVotes: 117
imdbID: tt0072728
--------------------------------------------------
The New Neighbor
1953
Retrieving the data of "the new neighbor" now... 
--------------------------------------------------
Title: The New Neighbor
Year: 1953
Rated: N/A
Released: 01 Aug 1953
Runtime: 7 min
Genre: Family, Com

Error:  Movie not found!
Shop Class
2020
Retrieving the data of "shop class" now... 
--------------------------------------------------
Title: Shop Class
Year: 2020
Rated: TV-G
Released: 28 Feb 2020
Runtime: 45 min
Genre: Adventure, Family, Game-Show
Director: N/A
Writer: N/A
Actors: Kelly Ractham, Brooks Atwood, Justin Long
Plot: It is a competition format that features teams of inventive students who are tasked with designing, building, and testing new contraptions.
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.1/10'}]
Metascore: N/A
imdbRating: 7.1
imdbVotes: 79
imdbID: tt10225108
--------------------------------------------------
Viking Warrior Women
2019
Retrieving the data of "viking warrior women" now... 
--------------------------------------------------
Title: Viking Warrior Women
Year: 2019
Rated: N/A
Released: 02 Mar 2020
Runtime: 51 min
Genre: Documentary, Drama, History
Director: David Bartlett, Sebastian P

...

The Incredible Dr. Pol: Blue Ribbon Kids
2016
Retrieving the data of "the incredible dr. pol: blue ribbon kids" now... 
the incredible
2016
--------2nd try: 2 words title --------
Error:  Movie not found!
Coco (in Spanish)
2017
Retrieving the data of "coco (in spanish)" now... 
coco (in
2017
--------2nd try: 2 words title --------
Error:  Movie not found!
Forky Asks a Question: What is Time?
2019
Retrieving the data of "forky asks a question: what is time?" now... 
forky asks
2019
--------2nd try: 2 words title --------
Error:  Movie not found!
The Wonderful World of Disney Presents The Little Mermaid Live!
2019
Retrieving the data of "the wonderful world of disney presents the little mermaid live!" now... 
the wonderful
2019
--------2nd try: 2 words title --------
Error:  Movie not found!
Forky Asks a Question: What is Art?
2019
Retrieving the data of "forky asks a question: what is art?" now... 
forky asks
2019
--------2nd try: 2 words title --------
Error:  Movie not found!
Mickey M

America's Heart & Soul
2004
Retrieving the data of "america's heart & soul" now... 
--------------------------------------------------
Title: America's Heart & Soul
Year: 2004
Rated: PG
Released: 02 Jul 2004
Runtime: 84 min
Genre: Documentary
Director: Louie Schwartzberg
Writer: N/A
Actors: George Woodard, Charles Jimmie Sr., The Vasquez Brothers
Plot: Filmmaker Louis Schwartzberg hits the road to capture America's people and its natural beauty.
Language: English
Country: United States
Awards: 2 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.0/10'}, {'Source': 'Metacritic', 'Value': '55/100'}]
Metascore: 55
imdbRating: 5.0
imdbVotes: 679
imdbID: tt0381006
--------------------------------------------------
An Extremely Goofy Movie
2000
Retrieving the data of "an extremely goofy movie" now... 
--------------------------------------------------
Title: An Extremely Goofy Movie
Year: 2000
Rated: G
Released: 29 Feb 2000
Runtime: 79 min
Genre: Animation, Comedy, Famil

Bambi II
2006
Retrieving the data of "bambi ii" now... 
bambi ii
2006
--------2nd try: 2 words title --------
Error:  Movie not found!
Bao
2018
Retrieving the data of "bao" now... 
--------------------------------------------------
Title: Bao
Year: 2018
Rated: G
Released: 15 Jun 2018
Runtime: 8 min
Genre: Animation, Short, Drama
Director: Domee Shi
Writer: Domee Shi
Actors: Sindy Lau, Daniel Kai Lin, Sharmaine Yeoh
Plot: A Chinese-Canadian woman suffering from empty nest syndrome gets a second shot at motherhood when one of her handmade dumplings comes alive.
Language: None
Country: United States
Awards: Won 1 Oscar. 1 win & 4 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '89%'}]
Metascore: N/A
imdbRating: 7.5
imdbVotes: 26,063
imdbID: tt8075496
--------------------------------------------------
Beach Picnic
1939
Retrieving the data of "beach picnic" now... 
--------------------------------------------------

Bone Bandit
1948
Retrieving the data of "bone bandit" now... 
--------------------------------------------------
Title: Bone Bandit
Year: 1948
Rated: Approved
Released: 30 Apr 1948
Runtime: 7 min
Genre: Family, Comedy, Animation
Director: Charles A. Nichols, Clyde Geronimi, Jack Kinney
Writer: Art Scott, Sterling Sturtevant
Actors: Pinto Colvig
Plot: Pluto's efforts to dig up a bone are frustrated by his allergy to goldenrod, and by a gopher who is using the bone to prop up his tunnel.
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.3/10'}]
Metascore: N/A
imdbRating: 6.3
imdbVotes: 540
imdbID: tt0040181
--------------------------------------------------
Bonkers
1993
Retrieving the data of "bonkers" now... 
--------------------------------------------------
Title: Bonkers
Year: 1993–1994
Rated: TV-Y
Released: 04 Sep 1993
Runtime: 27 min
Genre: Animation, Short, Action
Director: N/A
Writer: Jay Lender
Actors: Jim Cummings, 

Camp Rock 2: The Final Jam
2010
Retrieving the data of "camp rock 2: the final jam" now... 
--------------------------------------------------
Title: Camp Rock 2: The Final Jam
Year: 2010
Rated: TV-G
Released: 03 Sep 2010
Runtime: 97 min
Genre: Comedy, Family, Music, Musical, Romance
Director: Paul Hoen
Writer: Dan Berendsen, Karin Gist, Regina Y. Hicks, Karin Gist (characters), Regina Y. Hicks (characters), Julie Brown (characters), Paul Brown (characters)
Actors: Demi Lovato, Joe Jonas, Nick Jonas, Kevin Jonas
Plot: Mitchie is back with her friends at Camp Rock, ready to perform music, dance and have a good time. Her "boyfriend" is there as well. A new camp has opened across the lake, creating an atmosphere of competition or feud.
Language: English
Country: USA
Awards: 2 wins & 3 nominations.
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.2/10'}, {'Source': 'Rotten Tomatoes', 'Value': '63%'}]
Metascore: N/A
imdbRating: 5.2
imdbVotes: 17,122
imdbID: tt1252380
------------

In [13]:
#SECOND HALF QUERY
#query for all the tv shows in our disney list and get the latest information from IMDB
#half the list due to 1000 calls / day limit

for i in range(700,len(disney_df)): 
    title = disney_df['movie_title'][i].lower()
    year = disney_df['year'][i]
    print (disney_df['movie_title'][i])
    print (disney_df['year'][i])
    print ('=====================================')
    search_movie(title,year)
    time.sleep(10)

Clock Cleaners
1937
Retrieving the data of "clock cleaners" now... 
--------------------------------------------------
Title: Clock Cleaners
Year: 1937
Rated: Approved
Released: 15 Oct 1937
Runtime: 9 min
Genre: Animation, Short, Comedy
Director: Ben Sharpsteen
Writer: Otto Englander, Roy Williams
Actors: Pinto Colvig, Walt Disney, James MacDonald
Plot: Mickey, Donald, and Goofy are cleaning a large clock. Among the complications: Mickey fights a sleeping stork that doesn't want to leave, Donald gets tangled up in the main-spring, and Goofy is inside the bell when the clock strikes 
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.5/10'}]
Metascore: N/A
imdbRating: 7.5
imdbVotes: 2,489
imdbID: tt0028726
--------------------------------------------------
Cloud 9
2014
Retrieving the data of "cloud 9" now... 
--------------------------------------------------
Title: Cloud 9
Year: 2014
Rated: TV-G
Released: 17 Jan 2014
Runtime

Day & Night
2010
Retrieving the data of "day & night" now... 
--------------------------------------------------
Title: Day & Night
Year: 2010
Rated: G
Released: 18 Jun 2010
Runtime: 6 min
Genre: Animation, Short, Comedy
Director: Teddy Newton
Writer: Teddy Newton, Karen Paik
Actors: Muneer Lyati, Wayne Dyer
Plot: Day encounters Night, a stranger of distinctly darker moods. But their suspicions soon turn to curiosity, and they are delighted to find the joys of their budding friendship.
Language: English
Country: United States
Awards: Nominated for 1 Oscar. 3 wins & 1 nomination total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '8.0/10'}]
Metascore: N/A
imdbRating: 8.0
imdbVotes: 18,401
imdbID: tt1620446
--------------------------------------------------
Decorating Disney: Holiday Magic
2017
Retrieving the data of "decorating disney: holiday magic" now... 
--------------------------------------------------
Title: Decorating Disney: Holiday Magic
Year: 2017
Rated: N/A
Releas

Error:  Movie not found!
Disney K.C. Undercover
2014
Retrieving the data of "disney k.c. undercover" now... 
disney k.c.
2014
--------2nd try: 2 words title --------
Error:  Movie not found!
Disney Lab Rats: Elite Force
2015
Retrieving the data of "disney lab rats: elite force" now... 
disney lab
2015
--------2nd try: 2 words title --------
Error:  Movie not found!
Disney Legend of the Three Caballeros
2017
Retrieving the data of "disney legend of the three caballeros" now... 
disney legend
2017
--------2nd try: 2 words title --------
Error:  Movie not found!
Disney Mickey Mouse Clubhouse
2005
Retrieving the data of "disney mickey mouse clubhouse" now... 
disney mickey
2005
--------2nd try: 2 words title --------
Error:  Movie not found!
Disney Milo Murphy's Law
2016
Retrieving the data of "disney milo murphy's law" now... 
disney milo
2016
--------2nd try: 2 words title --------
Error:  Movie not found!
Disney Minnie's Bow-Toons (Shorts)
2011
Retrieving the data of "disney minnie's bo

Dog Whisperer with Cesar Millan
2004
Retrieving the data of "dog whisperer with cesar millan" now... 
--------------------------------------------------
Title: Dog Whisperer with Cesar Millan
Year: 2004–2016
Rated: TV-G
Released: 13 Sep 2004
Runtime: 110 min
Genre: Family, Reality-TV
Director: N/A
Writer: N/A
Actors: Cesar Millan, Daddy, Paul Dini
Plot: Dog lover Cesar Millan has an uncanny ability to rehabilitate problem dogs of all shapes and sizes.
Language: English
Country: United States
Awards: Nominated for 3 Primetime Emmys. 2 wins & 5 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '8.0/10'}]
Metascore: N/A
imdbRating: 8.0
imdbVotes: 2,731
imdbID: tt0423642
--------------------------------------------------
Don't Look Under the Bed
1999
Retrieving the data of "don't look under the bed" now... 
--------------------------------------------------
Title: Don't Look Under the Bed
Year: 1999
Rated: TV-PG
Released: 09 Oct 1999
Runtime: 100 min
Genre: Family,

Eddie's Million Dollar Cook-off
2003
Retrieving the data of "eddie's million dollar cook-off" now... 
--------------------------------------------------
Title: Eddie's Million Dollar Cook-Off
Year: 2003
Rated: TV-G
Released: 18 Jul 2003
Runtime: 110 min
Genre: Comedy, Family, Sport
Director: Paul Hoen
Writer: Jack Jason (story), Rick Bitzelberger (story), Dan Berendsen (story), Dan Berendsen (teleplay)
Actors: Taylor Ball, Orlando Brown, Reiley McClendon, Mark L. Taylor
Plot: Eddie loves baseball. His dad's the coach, his best friends are his teammates. But will he strike out with them all if he trades his glove for an oven mitt.
Language: English
Country: USA
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.1/10'}]
Metascore: N/A
imdbRating: 6.1
imdbVotes: 2,632
imdbID: tt0365149
--------------------------------------------------
Elena of Avalor (Shorts)
2017
Retrieving the data of "elena of avalor (shorts)" now... 
elena of
2017
--------2nd try: 2 words title -

...

Trail of the Panda
2009
Retrieving the data of "trail of the panda" now... 
trail of
2009
--------2nd try: 2 words title --------
Error:  Movie not found!
Trailer Horn
1950
Retrieving the data of "trailer horn" now... 
--------------------------------------------------
Title: Trailer Horn
Year: 1950
Rated: Approved
Released: 28 Apr 1950
Runtime: 6 min
Genre: Animation, Comedy, Family
Director: Jack Hannah
Writer: Roy Williams
Actors: Dessie Flynn, James MacDonald, Clarence Nash
Plot: Donald heads out for a nice relaxing camping trip, until he runs into trouble with Chip and Dale.
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.8/10'}]
Metascore: N/A
imdbRating: 6.8
imdbVotes: 664
imdbID: tt0043063
--------------------------------------------------
Treasure Island
1950
Retrieving the data of "treasure island" now... 
--------------------------------------------------
Title: Treasure Island
Year: 1950
Rated: PG
Released: 19

WALL-E
2008
Retrieving the data of "wall-e" now... 
--------------------------------------------------
Title: WALL·E
Year: 2008
Rated: G
Released: 27 Jun 2008
Runtime: 98 min
Genre: Animation, Adventure, Family
Director: Andrew Stanton
Writer: Andrew Stanton, Pete Docter, Jim Reardon
Actors: Ben Burtt, Elissa Knight, Jeff Garlin
Plot: In the distant future, a small waste-collecting robot inadvertently embarks on a space journey that will ultimately decide the fate of mankind.
Language: English, Hindi
Country: United States
Awards: Won 1 Oscar. 94 wins & 95 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '8.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '95%'}, {'Source': 'Metacritic', 'Value': '95/100'}]
Metascore: 95
imdbRating: 8.4
imdbVotes: 1,076,317
imdbID: tt0910970
--------------------------------------------------
Wendy Wu: Homecoming Warrior
2006
Retrieving the data of "wendy wu: homecoming warrior" now... 
-------------------------------------------

World's Greatest Dogs
2015
Retrieving the data of "world's greatest dogs" now... 
world's greatest
2015
--------2nd try: 2 words title --------
Error:  Movie not found!
X-Men: Evolution
2000
Retrieving the data of "x-men: evolution" now... 
--------------------------------------------------
Title: X-Men: Evolution
Year: 2000–2003
Rated: TV-Y7
Released: 04 Nov 2000
Runtime: 23 min
Genre: Animation, Action, Drama
Director: N/A
Writer: N/A
Actors: Scott McNeil, Meghan Black, Christopher Judge
Plot: This rendition of X-Men features Cyclops, Jean Grey, Rogue, Nightcrawler, Shadowcat and Spike as teenagers as they fight for a world that fears and hates them.
Language: English
Country: United States, Canada
Awards: 3 wins & 6 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.8/10'}]
Metascore: N/A
imdbRating: 7.8
imdbVotes: 11,659
imdbID: tt0247827
--------------------------------------------------
Ye Olden Days
1933
Retrieving the data of "ye olden days" now... 
-------

Zenon: Z3
2004
Retrieving the data of "zenon: z3" now... 
--------------------------------------------------
Title: Zenon: Z3
Year: 2004
Rated: TV-G
Released: 11 Jun 2004
Runtime: 81 min
Genre: Adventure, Comedy, Family, Sci-Fi
Director: Steve Rash
Writer: Marilyn Sadler (character from the book "Zenon: Girl of the Twenty First Century"), Roger Bollen (character from the book "Zenon: Girl of the Twenty First Century"), Stu Krieger
Actors: Kirsten Storms, Lauren Maltby, Alyson Morgan, Stuart Pankin
Plot: The heroine sets out to win an important competition - but her dreams of success are jeopardized.
Language: English
Country: USA
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.5/10'}]
Metascore: N/A
imdbRating: 5.5
imdbVotes: 2,499
imdbID: tt0379060
--------------------------------------------------
Brave Little Tailor
1938
Retrieving the data of "brave little tailor" now... 
--------------------------------------------------
Title: Brave Little Tailor
Year: 193

Eddie the Eagle
2016
Retrieving the data of "eddie the eagle" now... 
--------------------------------------------------
Title: Eddie the Eagle: Attitude Is Altitude - Filming the Ski Jumps
Year: 2016
Rated: N/A
Released: 14 Jun 2016
Runtime: 9 min
Genre: Documentary, Short, Sport
Director: N/A
Writer: N/A
Actors: Vic Armstrong, Dexter Fletcher, Hugh Jackman, David Reid
Plot: N/A
Language: English
Country: UK, Germany, USA
Awards: N/A
Ratings: []
Metascore: N/A
imdbRating: N/A
imdbVotes: N/A
imdbID: tt8115928
--------------------------------------------------
Bend It Like Beckham
2003
Retrieving the data of "bend it like beckham" now... 
--------------------------------------------------
Title: She Shoots, She Scores: The Making of 'Bend It Like Beckham'
Year: 2003
Rated: N/A
Released: N/A
Runtime: 15 min
Genre: Documentary, Short
Director: N/A
Writer: N/A
Actors: Gurinder Chadha, Simon Clifford, Trey Farley, Anupam Kher
Plot: N/A
Language: English
Country: USA
Awards: N/A
Ratings: [{'

In [14]:
disney_df.loc[disney_df['movie_title'] =='Paperman']

,show_id,type,movie_title,director,cast,country,date_added,year,rating,duration,listed_in,description
26,s30,Movie,Paperman,John Kahrs,"John Kahrs, Kari Wahlgren",United States,"November 12, 2021",2012,G,7 min,"Animation, Comedy, Family",A young man strives to find girl of his dreams.


## Print the content of the local database

In [15]:
print_database('output/IMDB_tv.sqlite')

('Ice Age: A Mammoth Christmas', '2011', 'TV-PG', '24 Nov 2011', 26, 'Animation, Short, Adventure, Comedy, Family, Fantasy', 'Karen Disher', 'Sam Harper, Mike Reiss', 'Ray Romano, John Leguizamo, Denis Leary, Queen Latifah', 'English', 'USA', '1 nomination.', -1.0, 6.5, '9,671', 'tt2100546')
('The Queen Family Singalong', '2021', 'TV-PG', '04 Nov 2021', 41, 'Music, Musical', 'Hamish Hamilton', 'Brian Strickland', 'Darren Criss, Adam Lambert, Jimmie Allen', 'English', 'United States', 'NA', -1.0, 5.9, '86', 'tt15716286')
('The Beatles: Get Back', '2021', 'PG-13', '25 Nov 2021', -1, 'Documentary, Biography, Music', 'Peter Jackson', 'NA', 'The Beatles, John Lennon, Paul McCartney', 'English', 'United Kingdom, New Zealand, United States', '4 wins & 4 nominations', -1.0, 9.1, '18,501', 'tt9735318')
('Becoming Cousteau', '2021', 'PG-13', '24 Nov 2021', 94, 'Documentary, Adventure, Biography', 'Liz Garbus', 'Mark Monroe, Pax Wassermann', 'Vincent Cassel, Carol Burnett, Louis Malle', 'English,

## Save the database content into an CSV file

In [16]:
save_in_csv('output/IMDB_tv.csv','output/IMDB_tv.sqlite')

In [17]:
#check the CSV is exported correctly
df=pd.read_csv('output/IMDB_tv.csv')
df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,Awards,Metascore,IMDBRating,imdbVotes,imdbID
0,Ice Age: A Mammoth Christmas,2011,TV-PG,24 Nov 2011,26,"Animation, Short, Adventure, Comedy, Family, F...",Karen Disher,"Sam Harper, Mike Reiss","Ray Romano, John Leguizamo, Denis Leary, Queen...",English,USA,1 nomination.,-1.0,6.5,"9,671",tt2100546
1,The Queen Family Singalong,2021,TV-PG,04 Nov 2021,41,"Music, Musical",Hamish Hamilton,Brian Strickland,"Darren Criss, Adam Lambert, Jimmie Allen",English,United States,NaN,-1.0,5.9,86,tt15716286
2,The Beatles: Get Back,2021,PG-13,25 Nov 2021,-1,"Documentary, Biography, Music",Peter Jackson,NaN,"The Beatles, John Lennon, Paul McCartney",English,"United Kingdom, New Zealand, United States",4 wins & 4 nominations,-1.0,9.1,"18,501",tt9735318
3,Becoming Cousteau,2021,PG-13,24 Nov 2021,94,"Documentary, Adventure, Biography",Liz Garbus,"Mark Monroe, Pax Wassermann","Vincent Cassel, Carol Burnett, Louis Malle","English, French",United States,Nominated for 1 BAFTA Film Award2 wins & 11 no...,74.0,7.1,834,tt10334438
4,Hawkeye,2021,TV-14,24 Nov 2021,60,"Action, Adventure, Crime",NaN,NaN,"Jeremy Renner, Hailee Steinfeld, Florence Pugh","English, American Sign",United States,8 nominations,-1.0,7.7,"142,549",tt10160804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,Aquamarine,2006,PG,03 Mar 2006,110,"Comedy, Family, Fantasy",Elizabeth Allen Rosenbaum,"John Quaintance, Jessica Bendinger, Alice Hoffman","Emma Roberts, JoJo, Sara Paxton",English,United States,1 win & 5 nominations,51.0,5.4,"41,611",tt0429591
807,X-Men Origins: Wolverine,2009,PG-13,01 May 2009,107,"Action, Sci-Fi",Gavin Hood,"David Benioff, Skip Woods","Hugh Jackman, Liev Schreiber, Ryan Reynolds",English,"United States, United Kingdom",3 wins & 21 nominations,40.0,6.6,"492,940",tt0458525
808,Night at the Museum: Battle of the Smithsonian,2009,PG,22 May 2009,105,"Adventure, Comedy, Family",Shawn Levy,"Robert Ben Garant, Thomas Lennon","Ben Stiller, Owen Wilson, Amy Adams",English,"United States, Canada",2 wins & 9 nominations,42.0,6.0,"191,332",tt1078912
809,Eddie the Eagle: Attitude Is Altitude - Filmin...,2016,NaN,14 Jun 2016,9,"Documentary, Short, Sport",NaN,NaN,"Vic Armstrong, Dexter Fletcher, Hugh Jackman, ...",English,"UK, Germany, USA",NaN,-1.0,-1.0,-1,tt8115928


In [18]:
df.count()

Title         811
Year          811
Rated         647
Released      798
Runtime       811
Genre         805
Director      579
Writer        612
Actors        776
Language      793
Country       801
Awards        407
Metascore     811
IMDBRating    811
imdbVotes     811
imdbID        811
dtype: int64

In [19]:
df[df.Title.str.startswith('Paper')]

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,Awards,Metascore,IMDBRating,imdbVotes,imdbID
16,Paperman,2012,G,02 Nov 2012,7,"Animation, Short, Comedy",John Kahrs,"Clio Chiang, Kendelle Hoyer","John Kahrs, Kari Wahlgren, Jack Goldenberg",English,United States,Won 1 Oscar. 3 wins & 2 nominations total,-1.0,8.2,"33,077",tt2388725
